# Importing libraries

In [1]:
%matplotlib inline
import os
import pandas as pd
from glob import glob
import numpy as np
import pickle
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as k
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib import figure
from keras.layers import Dense, Activation, Flatten, Dropout,BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers,optimizers
# import gc
# from path import Path

C:\Users\satish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# train data generator

In [7]:
from keras_preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255.)
train_generator = datagen.flow_from_directory(
directory= "D:/speech_dataset/train_images/",
target_size=(192,192),
#color_mode="rgb",
batch_size=32,
class_mode = "categorical",
shuffle=True,
seed=42
) 

Found 8700 images belonging to 29 classes.


# Validation data generator

In [5]:
#valid dataset
valid_generator = datagen.flow_from_directory(
directory= "D:/speech_dataset/val_images/",
target_size=(192,192),
#color_mode="rgb",
batch_size=32,
class_mode="categorical",
shuffle=True,
seed=42
)

Found 2900 images belonging to 29 classes.


# Test data generator

In [8]:
#test dataset
test_generator = datagen.flow_from_directory(
directory="D:/speech_dataset/test_data/",
target_size=(192,192),
#color_mode="rgb",
batch_size=1,
class_mode=None,
shuffle=False,
seed=42)

Found 9 images belonging to 1 classes.


# step size of the data

In [7]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_valid = valid_generator.n//valid_generator.batch_size

# Model(CNN)

In [9]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding="same",input_shape=(192,192,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64,(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64,(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(128,(3,3),padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(128,(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))
model.add(Dense(29,activation="softmax"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Model compilation

In [10]:
model.compile(optimizers.rmsprop(lr=0.001,decay=1e-6),
             loss="categorical_crossentropy",
             metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 192, 192, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 192, 192, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 190, 190, 64)      18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 190, 190, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 95, 95, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 95, 95, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 95, 95, 64)        36928     
__________

# Model fitting

In [11]:
#fitting our model
model.fit_generator(generator = train_generator,steps_per_epoch = step_size_train,
                    validation_data = valid_generator,validation_steps = step_size_valid,
                   epochs = 5)
model.evaluate_generator(generator = valid_generator,steps = step_size_valid)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
271/271 [==============================] - 4216s 16s/step - loss: 2.5047 - acc: 0.3017 - val_loss: 1.5804 - val_acc: 0.5448
Epoch 2/5
271/271 [==============================] - 3716s 14s/step - loss: 1.3315 - acc: 0.6119 - val_loss: 1.1128 - val_acc: 0.6778
Epoch 3/5
271/271 [==============================] - 4976s 18s/step - loss: 0.8266 - acc: 0.7564 - val_loss: 0.9415 - val_acc: 0.7500
Epoch 4/5
271/271 [==============================] - 4148s 15s/step - loss: 0.6035 - acc: 0.8263 - val_loss: 1.0474 - val_acc: 0.7622
Epoch 5/5
271/271 [==============================] - 4811s 18s/step - loss: 0.4756 - acc: 0.8624 - val_loss: 0.8467 - val_acc: 0.7842


[0.8455105931855312, 0.7841701535832766]

# Saving the model

In [18]:
pickle.dump(model,open('D:/final_project_model.pkl','wb'))

In [5]:
model=pickle.load(open('D:/final_model/final_project_model.pkl','rb'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


# Testing the model

In [9]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
#print(float(pred[0][0]))
labels = (train_generator.class_indices)
#print(labels)
predicted_class_indices=np.argmax(pred,axis=1)
predicted_class_indices[0:9]
percentage=[]
prediction=['bed','cat','five','happy','left','one','seven','sheila','wow']
labels=(train_generator.class_indices)
c=0
for i in prediction:
    index=labels[i]
    #print(index)
    percent=pred[c][index]
    #print(percent)
    percent=float(percent)
    percent=percent*100
    percentage.append(percent)
    c+=1
#Fetch labels from train gen for testing
#print(labels)
labels = dict((v,k) for k,v in labels.items())
#predictions = [labels[k] for k in predicted_class_indices]
for i in range(9):
    print(f'{prediction[i]} --> {percentage[i]}')
#print(predictions[0:9])

9/9 [==============================] - 3s 352ms/step
bed --> 3.8102060556411743
cat --> 0.4114062525331974
five --> 90.25172591209412
happy --> 98.97674918174744
left --> 52.98369526863098
one --> 31.525087356567383
seven --> 55.30551075935364
sheila --> 99.95682835578918
wow --> 19.99691128730774


In [ ]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
# test_generator.reset()
# pred=model.predict_generator(test_generator,
# steps=STEP_SIZE_TEST,
# verbose=1)
# pred[0]